In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3 as lite
import os
import sys
#import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
#%matplotlib inline
from collections import Counter
import itertools

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.124 Safari/537.36'}

In [3]:
cities = [[ 'tx' , 'austin'], [ 'ca' , 'los-angeles'], [ 'ga' , 'atlanta'] ,[ 'ma' , 'boston'] ,[ 'nc' , 'charlotte'] ,[ 'tx' , 'dallas'] ,[ 'co' , 'denver'] ,[ 'tx' , 'houston'] ,[ 'fl' , 'miami'] ,[ 'mn' , 'minneapolis'] ,[ 'ny' , 'new-york'] ,[ 'pa' , 'philadelphia'] ,[ 'az' , 'phoenix'] ,[ 'or' , 'portland'] ,[ 'ca' , 'san-francisco'] ,[ 'ca' , 'san-diego'] ,[ 'wa' , 'seattle'] ,[ 'dc' , 'washington'] ]

In [6]:
# CRAWL ALL THE PAGES OF INTERST
datadir = 'crawl/'
if not(os.path.isdir(datadir)):
    os.makedirs(datadir)

for city in cities:
    k = city[0]
    v = city[1]
    running = True
    page = 1
    print ''
    print k,v
    while running:
        url = "https://dogvacay.com/dog-boarding--" + k + "--" + v + "?p="+str(page)
        filename = datadir + k + '-' + v + '-' + str(page) + '.htm'
        if not(os.path.isfile(filename)):
            sys.stdout.write('-')
            r = requests.get(url, headers=headers)
            time.sleep(1)
            f = open(filename, 'w')
            f.write(r.text.encode('ascii', 'replace'))
            f.close()
            data = r.text
        else:
            sys.stdout.write('.')
            f = open(filename, 'r')
            data = f.read()
            f.close()
        soup = BeautifulSoup(data)
        pagination_links = soup.findAll('a', {'class': 'pagination-link'})
        running = False
        for pl in pagination_links:
            if pl.text.find('Next') == 0:
                running = True
                print url
                print pl
        page+=1



tx austin
.
ca los-angeles
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=1
<a class="pagination-link" data-page="2" href="/dog-boarding--ca--los-angeles?p=2">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=2
<a class="pagination-link" data-page="3" href="/dog-boarding--ca--los-angeles?p=3">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=3
<a class="pagination-link" data-page="4" href="/dog-boarding--ca--los-angeles?p=4">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=4
<a class="pagination-link" data-page="5" href="/dog-boarding--ca--los-angeles?p=5">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=5
<a class="pagination-link" data-page="6" href="/dog-boarding--ca--los-angeles?p=6">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=6
<a class="pagination-link" data-page="7" href="/dog-boarding--ca--los-angeles?p=7">Next »</a>
.https://dogvacay.com/dog-boarding--ca--los-angeles?p=7
<a class="pagina

In [104]:
pagination_links = soup.findAll('a', {'class': 'pagination-link'})
for pl in pagination_links:
            if pl.text.find('Next') == 0:
                print pl

In [ ]:
# Data extraction phase

times = []
fees = []
reviews = []
repeats = []
city = []
service = []
rating = []
name = []

soup = BeautifulSoup(data)
sitters = soup.findAll('div', {'class': 'dv-host-list-item'}) #replaced card-content
for sitter in sitters:
    rtwrap = sitter.findAll('div', {'class': 'icon-wrapper'})
    if len(rtwrap) > 0:
        response_time = rtwrap[0].text.strip('\n')[19:]
    else:
        response_time = -1
    fee = sitter.findAll('div', {'class': 'price price--primary u-text-center'})
    review = sitter.find('span', {'class': 'vcard-review'})
    if review is not None and len(review) > 0:
        review = int(review.text.replace('Reviews', '').replace('Review', '').strip())
    else:
        review = 0
    repeat = sitter.findAll('div', {'class': 'icon-wrapper'})
    if len(repeat) == 2 and len(repeat[1].text.strip()) > 12:
        repeat = repeat[1].text.strip()[:2]
    else:
        repeat = 0
    #services = sitter.findAll('div', {'class': 'vcard-line-item'})
    full_star = sitter.findAll('i', {'class': 'ck-star'})
    ratings = len(full_star)
    half_star = sitter.findAll('i', {'class': 'ck-star-half'})
    if len(half_star) > 0:
        ratings += .5
    names = sitter.find('h2', {'class': 'vcard-title'}).text.strip('\n$')#.split(' ')
    fees.append(fee[0].text.strip('\n$')[:2])
    title = soup.find('title').text
    i = title.find(' Dog Boarding')
    title = title[0:i]
    city.append(title)
    #times.append(response_time)
    reviews.append(review)
    repeats.append(repeat)
    #service.append(services[-1].text.strip('\n'))
    rating.append(ratings)
    name.append(names)

df = pd.DataFrame({'city': city, 'fee': fees, 'review' : reviews, 'repeat' : repeats, 'ratings': rating, 'names': name})    

In [33]:
df['names'] = df['names'].apply(lambda x: str(x).split())

In [34]:
len(sitters)

0

In [35]:
df

,city,fee,names,ratings,repeat,review


In [63]:
list_ = df['names'].tolist()
merged_list = list(itertools.chain(*list_))
Counter(merged_list)#.most_common(10)

Counter({'in': 2, 'home': 2, 'Lover': 2, 'Your': 2, 'City': 1, 'Holistic': 1, 'For': 1, 'Pet': 1, 'St.': 1, 'Paradise': 1, '2nd': 1, 'Bliss': 1, 'Hill': 1, 'Heavenly': 1, "Tuesday's": 1, 'Doggy': 1, 'Happy': 1, 'Pampering': 1, 'Petite': 1, 'Hotel': 1, 'unleashed': 1, 'Vernon': 1, 'setting': 1, "Brewer's": 1, 'Haven': 1, 'Retreat': 1, 'Care': 1, 'Vacay': 1, "pet's": 1, 'Poochie': 1, 'Mount': 1, 'Dog': 1, 'dogs!': 1, 'Dog-Sitting!': 1, 'tails': 1, 'Fun': 1, 'Doggies': 1, 'Square': 1, 'Love': 1, 'Labrador': 1, 'country': 1, 'Work-From-Home': 1, 'Johnston': 1, 'Bank': 1, 'Lifetime': 1, 'the': 1, 'Hound': 1, 'Galore!': 1})

In [59]:
words = df['names'].tolist()

In [60]:
Counter(word for line in words for word in line)

Counter({u' ': 37, u'e': 33, u'o': 28, u't': 20, u'r': 19, u'a': 18, u'i': 18, u'n': 18, u's': 14, u'g': 10, u'l': 9, u'u': 9, u'H': 8, u'd': 7, u'y': 7, u'h': 6, u'm': 6, u'L': 5, u'P': 5, u'v': 5, u'D': 4, u'c': 4, u'p': 4, u'!': 3, u"'": 3, u'-': 3, u'B': 3, u'F': 3, u'S': 3, u'C': 2, u'V': 2, u'Y': 2, u'k': 2, u'.': 1, u'2': 1, u'G': 1, u'J': 1, u'M': 1, u'R': 1, u'T': 1, u'W': 1, u'b': 1, u'f': 1, u'q': 1, u'w': 1})

In [290]:
df['names'][0]+df['names'][1]

[u'Summit', u'Dog', u'Inn', u"Holli's", u'House', u'For', u'Hounds!']

In [289]:
list = []
for row in (df['names']):
    row = list.append(row)
    print list[-1]

[u'Summit', u'Dog', u'Inn']
[u"Holli's", u'House', u'For', u'Hounds!']
[u'doggy', u'dogs', u'heaven']
[u'The', u'Bark', u'-', u'Dog', u'Destination', u'Vacation']
[u'The', u'Happy', u'Pack']
[u'Furry', u'Friends', u'Home', u'Away', u'From', u'Home']
[u'Lucky-Dog', u'Boarding']
[u'Pampered', u'Puppy', u'Paradise']
[u'Enter', u'as', u'Friends,', u'Leave', u'as', u'Family']
[u'WAGZ-N-WIGGLES']
[u'A', u"Pug's", u'Paradise']
[u'Cloud', u'K9']
[u'The', u'Farm']
[u'Reliable,', u'loving', u'care', u'for', u'your', u'dogs']
[u'I', u'love', u'all', u'Fur', u'Babies:)']


In [269]:
tuple(list)

([u'Summit', u'Dog', u'Inn'],
 [u"Holli's", u'House', u'For', u'Hounds!'],
 [u'doggy', u'dogs', u'heaven'],
 [u'The', u'Bark', u'-', u'Dog', u'Destination', u'Vacation'],
 [u'The', u'Happy', u'Pack'],
 [u'Furry', u'Friends', u'Home', u'Away', u'From', u'Home'],
 [u'Lucky-Dog', u'Boarding'],
 [u'Pampered', u'Puppy', u'Paradise'],
 [u'Enter', u'as', u'Friends,', u'Leave', u'as', u'Family'],
 [u'WAGZ-N-WIGGLES'],
 [u'A', u"Pug's", u'Paradise'],
 [u'Cloud', u'K9'],
 [u'The', u'Farm'],
 [u'Reliable,', u'loving', u'care', u'for', u'your', u'dogs'],
 [u'I', u'love', u'all', u'Fur', u'Babies:)'])

In [303]:
Counter(list)

TypeError: unhashable type: 'list'

In [129]:
a = np.array(df['names']).tolist()